In [1]:
import torch
import syft

In [2]:
hook = syft.TorchHook(torch)

In [3]:
alice = syft.VirtualWorker(hook,id="alice")
bob = syft.VirtualWorker(hook,id="bob")

In [4]:
Q_BITS = 7
field = 2**Q_BITS -1
def decompose(shares):
   """decompose a tensor into its binary representation."""
   powers = torch.arange(Q_BITS)
   locations = list(shares.keys())
   shares_shape = shares[locations[0]].shape

   for i in range(len(shares_shape)):
       powers = powers.unsqueeze(0)
   moduli = 2 ** powers

   decomposition = {}
   for location in locations:
       share = shares[location].unsqueeze(-1)
       moduli_ptr = moduli.send(location)

       decomposition[location] = torch.remainder(
           ((share + 2 ** (Q_BITS)) / moduli_ptr.type_as(share))
           , 2
       )
   return decomposition

In [5]:
t = torch.tensor([1, 2, 3, 4.0])
x = t.fix_prec(field = field, precision_fractional=1).share(bob,alice, field = field)

In [6]:
x

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:75654950657 -> bob:1901578229]
	-> (Wrapper)>[PointerTensor | me:26854439101 -> alice:26894443290]

In [7]:
print(alice._objects)

{26894443290: tensor([  0, 118,  63, 122])}


In [8]:
print(bob._objects)

{1901578229: tensor([10, 29, 94, 45])}


In [9]:
decomposed = decompose(x.child.child.child.child)

In [10]:
decomposed

{'bob': (Wrapper)>[PointerTensor | me:47256240258 -> bob:19756191051],
 'alice': (Wrapper)>[PointerTensor | me:89344671627 -> alice:27597645655]}

In [11]:
a= decomposed["alice"].get()

In [12]:
a

tensor([[0, 0, 0, 0, 0, 0, 0],
        [0, 1, 1, 0, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0],
        [0, 1, 0, 1, 1, 1, 1]])

In [13]:
b= decomposed["bob"].get()

In [14]:
b

tensor([[0, 1, 0, 1, 0, 0, 0],
        [1, 0, 1, 1, 1, 0, 0],
        [0, 1, 1, 1, 1, 0, 1],
        [1, 0, 1, 1, 0, 1, 0]])

In [15]:
def convert_to_int(bits):
    total = 0
    for i, bit in enumerate(bits):
        total += bit * 2**i
    return total

In [17]:
convert_to_int([0, 0, 0, 0, 0, 0, 0])

0

In [16]:
convert_to_int([0, 1, 0, 1, 0, 0, 0])

10

In [21]:
field

127

In [26]:
class test_base():
    def __init__(self):
        print(type(self))
class not_base(test_base):
    def __init__(self):
        print("testing")
        super().__init__()

test_base()

<class '__main__.test_base'>


In [18]:
x = torch.tensor([1,2,3,4])
x.type()

'torch.LongTensor'